# Chapter 8 - Land Data Example
### Calculate changes in vegetation extent in a given area

In In this chapter we plot and calculate changes in vegetation extent in a selected area, using MODIS data

In [ ]:
# libraries
import numpy as np
import pandas as pd
import xarray as xr
import os
import re
import urllib as ur
import requests
import matplotlib.pyplot as plt 
import hvplot.pandas
import hvplot.xarray
xr.set_options(display_style="html")  #display dataset nicely

# this library helps to make your code execution less messy
import warnings
warnings.simplefilter('ignore') # filter some warning messages

In [ ]:
# parameters
# region
lat1, lat2 = 16, 18
lon1, lon2 = -98,-96
# all time available
mon = 8 # month
dy = 15 # day

In [ ]:
# find each year image (same day)
ayrs = [*range(1989,2021)]
iurl = 'https://www.ncei.noaa.gov/thredds/fileServer/' # server link
# looks for fle each year
flist = list()

for i in ayrs:
    r = requests.get('https://www.ncei.noaa.gov/thredds/catalog/cdr/ndvi/files/'+str(i)+'/catalog.html')   
    #print(r.text)
    x = re.split("\'",r.text)
    for j in x:
        #if (".nc" in i) & ("catalog.html?" in i):
        dtnm = '_'+str(i)+str(mon).zfill(2)+str(dy).zfill(2)+'_'
        if (dtnm in j) & ("catalog.html?" in j):
            flist.append(j)

In [ ]:
tdt = list()
for iy, i in enumerate(flist):
    iurl2 = iurl + re.split('=',i)[1]
    print(iy,i)
    tmp = ur.request.urlretrieve(iurl2, 'tmp.nc')
    nvi = xr.open_dataset('tmp.nc')
    #print(nvi)
    nvi.close()
    tdt.append(nvi.time.data[0])
    tmp1 = nvi.NDVI[0,:,:].sel(latitude=slice(lat2,lat1),longitude=slice(lon1,lon2))
    if iy==0:
        nvi_dt = np.full((len(flist),tmp1.shape[0],tmp1.shape[1]), np.nan)
    nvi_dt[iy,:,:] = tmp1.data
    os.remove('tmp.nc')

In [ ]:
ndvi = xr.DataArray(data=nvi_dt, dims=['time','latitude','longitude'], 
                    coords=dict(time=tdt,latitude=tmp1.latitude.values, longitude=tmp1.longitude.values),attrs=tmp1.attrs) 
ndvi

In [ ]:
mean_ndvi = ndvi.mean(axis=0)
mean_ndvi.plot(cmap='YlGn')

In [ ]:
(ndvi.sel(time='2018-'+str(mon).zfill(2)+'-'+str(dy).zfill(2))-mean_ndvi).plot()

In [ ]:
veg_area = list()
veg_mean = list()
for i in range(len(ayrs)): 
    tmp=ndvi[i,:,:].where(ndvi[i,:,:]>=0.03)
    veg_area.append(tmp.count())
    veg_mean.append(tmp.mean())
plt.bar(ayrs,veg_area-np.nanmean(veg_area))
plt.show()
plt.bar(ayrs,veg_mean-np.nanmean(veg_mean))
plt.show()

In [ ]:
# another example

In [ ]:
iurl = 'https://data.giss.nasa.gov/pub/gistemp/gistemp250_GHCNv4.nc.gz'
#r = requests.get(iurl)
with open('gistmep.nc.gz', "wb") as f:
    r = requests.get(iurl)
    f.write(r.content)
f.close()
os.system('gunzip gistmep.nc.gz')
tas = xr.open_dataset('gistmep.nc')
tas.tempanomaly.sel(lat=17,lon=-97,time=slice('1980-01-01','2021-08-15')).plot()
plt.grid()
tas